In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from google.cloud import storage
from firebase_admin import auth
from firebase_admin import firestore

c:\Programming\Software\Python\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
user_email = "mokiczena@gmail.com"

json_name = 'my-speechy-firebase-adminsdk-g7v3g-48589351f8.json'
cred = credentials.Certificate(json_name)

default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://my-speechy-default-rtdb.europe-west1.firebasedatabase.app/'
})

In [46]:
storage_client = storage.Client.from_service_account_json(json_name)
firestore_client = firestore.client()
bucket = storage_client.get_bucket('my-speechy.appspot.com')
user = auth.get_user_by_email(user_email)
user_id = user.uid

In [35]:
def remove_profile_pics():
    profile_pic_exists = db.reference(f'users/{user_id}/profilePicUpdated').get()
    if (profile_pic_exists is not None):
        for quality in ('normalQuality', 'lowQuality'):
            try:
                bucket.blob(f'profilePics/{user_id}/{quality}/{user_id}.jpg').delete()
            except Exception as e:
                print(e)

In [36]:
def get_other_user_id(chat_id):
    return next(split for split in chat_id.split('_') if split != user_id)

def leave_private_chat_use_case(chat_id):
    other_user_id = get_other_user_id(chat_id)
    other_user_chat = db.reference(f'users/{other_user_id}/private_chats/{chat_id}').get()
    if (other_user_chat is None):
        db.reference(f'messages/{chat_id}').delete()
    db.reference(f'users/{user_id}/private_chats/{chat_id}').delete()
    db.reference(f'private_chats/{user_id}/{chat_id}').delete()

def check_if_is_admin_use_case(chat_id):
    return db.reference(f'admins/{chat_id}').get() == user_id

def decrement_member_count_use_case(chat_id, remove):
    ref = db.reference(f'member_count/{chat_id}')
    member_count = ref.get()
    if (member_count is None): return
    if (member_count - 1 <= 0 or remove): ref.delete()
    else: ref.set(member_count-1)

def delete_public_chat_use_case(chat_id):
    db.reference(f'messages/{chat_id}').delete()
    decrement_member_count_use_case(chat_id, True)
    db.reference(f'public_chats/{chat_id}').delete()
    db.reference(f'members/{chat_id}').delete()
    db.reference(f'admins/{chat_id}').delete()

def leave_public_chat_use_case(chat_id, revoke):
    if (revoke):
        decrement_member_count_use_case(chat_id, False)
        db.reference(f'members/{chat_id}/{user_id}').delete()
    db.reference(f'users/{user_id}/public_chats/{chat_id}').delete()


def revoke_membership():
    user_path = f'users/{user_id}'
    private_chats = db.reference(f'{user_path}/private_chats').get()
    if (private_chats is not None):
        for private_chat in private_chats:
            leave_private_chat_use_case(private_chat)
    public_chats = db.reference(f'{user_path}/public_chats').get()
    if (public_chats is not None):
        for public_chat in public_chats:
            isAdmin = check_if_is_admin_use_case(public_chat)
            if (isAdmin): delete_public_chat_use_case(public_chat)
            leave_public_chat_use_case(public_chat, not isAdmin)

In [37]:
def delete_firestore_data():
    user_ref = firestore_client.collection('users').document(user_id)
    for coll in ('lessons', 'meditation', 'thoughtTracks'):
        items_ref = user_ref.collection(coll)
        docs = items_ref.get()
        if docs is not None:
            for doc in docs:
                items_ref.document(doc.id).delete()

In [38]:
def delete_rdb_user():
    db.reference(f'users/{user_id}').delete()

In [39]:
def delete_user():
    auth.delete_user(user_id)

In [44]:
def deletion_service():
    remove_profile_pics()
    revoke_membership()
    delete_firestore_data()
    delete_rdb_user()
    delete_user()

In [47]:
deletion_service()